# Implementing and testing the equatorial to Galactocentric transformation in Astropy

https://docs.astropy.org/en/stable/coordinates/galactocentric.html#coordinates-galactocentric

In [618]:
%matplotlib inline

In [619]:
import numpy as np
import matplotlib.pyplot as plt


import astropy.stats as aps
import astropy.coordinates as coord
import astropy.units as u
from astropy.coordinates.builtin_frames.galactocentric \
    import get_matrix_vectors
import warnings
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)

from astropy.coordinates import Galactocentric
Galactocentric()

<Galactocentric Frame (galcen_coord=<ICRS Coordinate: (ra, dec) in deg
    (266.4051, -28.936175)>, galcen_distance=8.3 kpc, galcen_v_sun=(11.1, 232.24, 7.25) km / s, z_sun=27.0 pc, roll=0.0 deg)>

In [620]:
# Helper function for converting degrees to radians.
def deg_to_rad(deg):
    # rads / 2pi = degrees / 360
    return deg * (2 * np.pi) / 360

def rad_to_deg(rad):
    return rad * 360 / (2 * np.pi)

assert deg_to_rad(0) == 0
assert deg_to_rad(90)*2 == np.pi
assert deg_to_rad(180) == np.pi
assert deg_to_rad(360) == 2*np.pi

th1, th2 = deg_to_rad(np.array([0, 90]))
assert th1 == 0
assert th2*2 == np.pi 

Constants

In [621]:
ra_gc_deg, dec_gc_deg = 266.4051, -28.936175

ra_gc, dec_gc = deg_to_rad(np.array([ra_gc_deg, dec_gc_deg]))

eta_deg = 58.5986320306
eta = deg_to_rad(eta_deg)

d_gc = 8.3
zsun = 27.*1e-3

Test star parameters.

In [622]:
nstars = 1
ra = ra_gc_deg
dec = dec_gc_deg
d = 1 # kpc

Convert to cartesian coordinates.

In [628]:
#  Rotation matrix to convert to cartesian.

def r_icrs(ra_deg, dec_deg, d):
    ra, dec = deg_to_rad(np.array([ra_deg, dec_deg]))
    return np.array([
        [d * np.cos(ra) * np.cos(dec)],
        [d * np.sin(ra) * np.cos(dec)],
        [d * np.sin(dec)]
    ])

r = r_icrs(ra, dec, d)
print(r_icrs(0, 20, 1.5))

assert np.shape(r)[0] == 3
assert np.shape(r)[1] == nstars

[[1.40953893]
 [0.        ]
 [0.51303021]]


In /Users/rangus/Applications/anaconda3/lib/python3.7/site-packages/astropy/coordinates/builtin_frames/galactocentric.py:

In [629]:
# <Galactocentric Frame (galcen_coord=<ICRS Coordinate: (ra, dec) in deg
#     (266.4051, -28.936175)>, galcen_distance=8.122 kpc, galcen_v_sun=(12.9, 245.6, 7.78) km / s, z_sun=20.8 pc, roll=0.0 deg)>

R Rotates the cartesian frame so that x points towards the galactic center and y is in line with the Galactic plane.

In [630]:
# Rotation matrices to point the x-axis towards the Galactic center.

R1 = np.array([
    [np.cos(dec_gc), 0, np.sin(dec_gc)],
    [0, 1, 0],
    [-np.sin(dec_gc), 0, np.cos(dec_gc)]
])

R2 = np.array([
    [np.cos(ra_gc), np.sin(ra_gc), 0],
    [-np.sin(ra_gc), np.cos(ra_gc), 0],
    [0, 0, 1]
])

assert np.shape(R1) == (3, 3)
assert np.shape(R2) == (3, 3)

In [631]:
R1_R2 = np.dot(R1, R2)
print(np.dot(R1_R2, r))

[[1.00000000e+00]
 [0.00000000e+00]
 [5.55111512e-17]]


In [632]:
# Now align with the plane of the Galaxy, with roll angle eta.

R3 = np.array([
    [1, 0, 0],
    [0, np.cos(eta), np.sin(eta)],
    [0, -np.sin(eta), np.cos(eta)]
])

assert np.shape(R3) == (3, 3)

In [633]:
# R = R3 . R1 . R2

R3_R1 = np.dot(R3, R1)
Rtest = np.dot(R3_R1, R2)

R1_R2 = np.dot(R1, R2)
R = np.dot(R3, R1_R2)

# for i in range(3):
#     for j in range(3):
#         assert np.isclose(R[i, j], Rtest[i, j], atol=1e-10)
# assert np.shape(R) == (3, 3)

# R = np.dot(R1_R2, R3)
R = np.dot(R3, R1_R2)
# R = np.dot(R3_R1, R2)

print(np.dot(R, r))

[[1.]
 [0.]
 [0.]]


In [634]:
def full_R():
    return np.array([
        [np.cos(ra_gc)*np.cos(dec_gc), np.cos(dec_gc)*np.sin(ra_gc), -np.sin(dec_gc)],
        [np.cos(ra_gc)*np.sin(dec_gc)*np.sin(eta) - np.sin(ra_gc)*np.cos(eta), np.sin(ra_gc)*np.sin(dec_gc)*np.sin(eta)+np.cos(ra_gc)*np.cos(eta), np.cos(dec_gc)*np.sin(eta)],
        [np.cos(ra_gc)*np.sin(dec_gc)*np.cos(eta) + np.sin(ra_gc)*np.sin(eta), np.sin(ra_gc)*np.sin(dec_gc)*np.cos(eta) - np.cos(ra_gc)*np.sin(eta), np.cos(dec_gc)*np.cos(eta)]
    ])

In [635]:
print(np.dot(R, r))

[[1.]
 [0.]
 [0.]]


Subtract the distance to the Galactic center.

In [636]:
xhat = np.array([[1, 0, 0]]).T

R_r = np.dot(R, r)
rdash = R_r - (d_gc * xhat)
print(rdash)
print(R_r)

[[-7.3]
 [ 0. ]
 [ 0. ]]
[[1.]
 [0.]
 [0.]]


Account for Sun's height above the midplane.

In [637]:
theta = np.arcsin(zsun/d_gc)

H = np.array([
    [np.cos(theta), 0, np.sin(theta)],
    [0, 1, 0],
    [-np.sin(theta), 0, np.cos(theta)]
])

rgc = np.dot(H, rdash)
print("x = ", rgc[0][0]*u.kpc)
print("y = ", rgc[1][0]*u.kpc)
print("z = ", rgc[2][0]*u.kpc)

x =  -7.29996137527886 kpc
y =  0.0 kpc
z =  0.02374698795180723 kpc


Put it all together.

In [638]:
def eqtogal(ra, dec, d):
    
    r = r_icrs(ra, dec, d)

    R1 = np.array([[np.cos(dec_gc), 0, np.sin(dec_gc)],
                   [0, 1, 0],
                   [-np.sin(dec_gc), 0, np.cos(dec_gc)]])
    R2 = np.array([[np.cos(ra_gc), np.sin(ra_gc), 0],
                   [-np.sin(ra_gc), np.cos(ra_gc), 0],
                   [0, 0, 1]])
    R3 = np.array([[1, 0, 0],
                   [0, np.cos(eta), np.sin(eta)],
                   [0, -np.sin(eta), np.cos(eta)]])
    
    R1_R2 = np.dot(R1, R2)
    R = np.dot(R3, R1_R2)
    
    xhat = np.array([[1, 0, 0]]).T
    R_r = np.dot(R, r)
    rdash = R_r - d_gc * xhat
    
    theta = np.arcsin(zsun/d_gc)
    H = np.array([[np.cos(theta), 0, np.sin(theta)],
                  [0, 1, 0],
                  [-np.sin(theta), 0, np.cos(theta)]])
    
    rgc = np.dot(H, rdash)
    return rgc

In [639]:
rgc = eqtogal(ra, dec, d)
print("x = ", rgc[0][0]*u.kpc)
print("y = ", rgc[1][0]*u.kpc)
print("z = ", rgc[2][0]*u.kpc)

x =  -7.29996137527886 kpc
y =  0.0 kpc
z =  0.02374698795180723 kpc


Compare with Astropy.

In [640]:
galcen_frame = coord.Galactocentric()

# Calculate XYZ position from ra, dec and parallax
c = coord.SkyCoord(ra = ra*u.deg,
                   dec = dec*u.deg,
                   distance = d*u.kpc)
galcen = c.transform_to(galcen_frame)

print("x = ", galcen.data.xyz[0])
print("y = ", galcen.data.xyz[1])
print("z = ", galcen.data.xyz[2])

x =  -7.299961375278859 kpc
y =  0.0 kpc
z =  0.023746987951807217 kpc


Code for converting km/s to mas/yr.

In [661]:
def cartesian_to_angular(km_s, kpc):
    to_km_yr = (365.2425 * 24*3600)*u.s/u.yr
    to_m_yr = 1000*u.m/u.km
    to_kpc_yr = 1./(const.kpc/u.kpc)
    to_rad_yr = u.rad/u.kpc/kpc
    to_deg_yr = 360*u.deg/(2*np.pi*u.rad)
    to_mas_yr = 3600*1000*u.mas/u.deg
    return np.arcsin(((km_s * to_km_yr) * to_m_yr * to_kpc_yr * to_rad_yr).value) *u.rad/u.yr * to_deg_yr * to_mas_yr

def cartesian_to_angular_no_units(km_s, kpc):
    to_km_yr = 365.2425 * 24*3600
    to_m_yr = 1000
    to_kpc_yr = 1./3.0856775814671917e+19
    to_rad_yr = 1./kpc
    to_deg_yr = 360/(2*np.pi)
    to_mas_yr = 3600*1000
    return np.arcsin((km_s * to_km_yr) * to_m_yr * to_kpc_yr * to_rad_yr) * to_deg_yr * to_mas_yr

Test

In [662]:
import astropy.constants as const

true_pm = 20 * u.mas/u.yr
distance = 5
pm = true_pm*1

# Convert to degrees/yr
pm /= ((3600*1000)*u.mas/u.deg)

# Convert to radians/yr
pm *= 2*np.pi/360 * u.rad/u.deg

# Convert to distance.
pm *= distance*u.kpc /u.rad

# Convert to m/yr
pm *= const.kpc/u.kpc

# convert to km/yr
pm *= 1e-3*u.km/u.m

# convert to km/s
yr = 24*60*60*365.25
pm /= yr*u.s/u.yr

pm

<Quantity 474.04704635 km / s>

In [663]:
print(cartesian_vel_to_angular_vel(pm, distance))
print(cartesian_to_angular_no_units(pm.value, distance))

19.99958932238196 mas / yr
19.99958932238196


In [664]:
assert np.isclose((pm / (distance*u.kpc)).to(u.mas/u.yr, u.dimensionless_angles()), true_pm, atol=1e-10)
assert np.isclose(cartesian_to_angular_no_units(pm.value, distance), true_pm.value, atol=1e-3)
assert np.isclose(cartesian_to_angular(pm, distance), true_pm, atol=1e-3)